# Random Graph Analysis: ER and BA Networks

**Purpose**: Compute strategy correlation coefficients (θ, φ) for multilayer random networks  
**Graph Types**: Erdős-Rényi (ER) and Barabási-Albert (BA)  
**Update Rules**: Death-Birth (dB) and Fitness-Proportional (ff)

---

## Overview

This notebook extends the multilayer EGT framework to random graph ensembles. It sweeps over:
- **ER graphs**: Connection probability p ∈ [0.1, 0.5]
- **BA graphs**: Preferential attachment parameter k ∈ [1, 5]

For each graph pair, all N² initial cooperator placements are evaluated.

### Dependencies
Core functions (`bc_multilayer_DB`, `beta_gamma`, etc.) are imported from `multilayer_egt.ipynb`.

---
## 1. Configuration

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Graph type: :ER (Erdős-Rényi) or :BA (Barabási-Albert)
GRAPH_TYPE = :ER

# Update rule: :dB (death-Birth) or :ff (fitness-proportional)
UPDATE_RULE = :dB

# Network size
N = 15

# Parameter ranges
ER_PROBS = [0.1, 0.2, 0.3, 0.4, 0.5]     # Connection probabilities
BA_K_VALUES = [1, 2, 3, 4, 5]             # Edges per new node

# Simulation settings
NUM_SEEDS = 5          # Graph instances per parameter
CHECKPOINT_EVERY = 50  # Save frequency

println("Config: $(GRAPH_TYPE)-$(UPDATE_RULE), N=$(N)")

In [ ]:
using Random, Graphs, Roots, Dates, Serialization
using Printf, Statistics, LinearAlgebra, DelimitedFiles
using IterativeSolvers, SparseArrays, DataFrames

---
## 2. Core Functions (Shared with multilayer_egt.ipynb)

These functions compute the strategy correlation coefficients. They are identical to those in `multilayer_egt.ipynb` and included here for standalone execution.

In [ ]:
# -----------------------------------------------------------------------------
# NOTE: These core functions are shared with multilayer_egt.ipynb
# If refactoring, extract to a shared module: MultilayerEGT.jl
# -----------------------------------------------------------------------------

"""Rearrange edge sequence by layer and node order."""
function edge_seq_rearrangement(edge_seq::Array{Int64,2})
    layer_list = setdiff(edge_seq[:,4], [])
    L = maximum(layer_list)
    N = maximum(edge_seq[:,1])
    
    edge_seq_rearranged = zeros(Int64, size(edge_seq))
    id = 1
    for ell in layer_list
        edge_list_ell = findall(isequal(ell), edge_seq[:,4])
        M = sparse(edge_seq[edge_list_ell,1], edge_seq[edge_list_ell,2], edge_seq[edge_list_ell,3])
        inds = findall(!iszero, M)
        a, b = getindex.(inds, 1), getindex.(inds, 2)
        len = length(a)
        edge_seq_rearranged[id:id+len-1, :] = hcat(b, a, M[inds], fill(ell, len))
        id += len
    end
    return edge_seq_rearranged
end

"""Solve for β and γ coefficients via iterative linear solver."""
function beta_gamma(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    N = maximum(edge_seq[:,1])
    L = maximum(edge_seq[:,4])
    
    # Build transition matrix
    M = spzeros(Float64, N*L, N)
    for ell = 1:L
        edge_list_ell = findall(isequal(ell), edge_seq[:,4])
        M[(ell-1)*N+1:ell*N, :] = sparse(edge_seq[edge_list_ell,1], edge_seq[edge_list_ell,2], 
                                          edge_seq[edge_list_ell,3], N, N)
    end
    
    # Compute stationary distribution
    pi = spzeros(Float64, L*N)
    for ell = 1:L
        pi_2 = sum(M[(ell-1)*N+1:ell*N, :], dims=2)[:,1]
        presence = findall(!iszero, pi_2)
        pi_transient = spzeros(Float64, N)
        pi_transient[presence] = pi_2[presence] / sum(pi_2)
        pi[(ell-1)*N+1:ell*N] = pi_transient
        M_transient = M[(ell-1)*N+1:ell*N, :]
        M_transient[presence, :] ./= pi_2[presence]
        M[(ell-1)*N+1:ell*N, :] = M_transient
    end

    # Strategy configuration
    xi = spzeros(Float64, L*N)
    for ell = 1:L
        presence = findall(isequal(ell), xi_seq[:,3])
        xi_t = spzeros(Int64, N)
        xi_t[xi_seq[presence,1]] = xi_seq[presence,2]
        xi[(ell-1)*N+1:ell*N] = xi_t
    end

    # Solve for β_i
    presence_list = findall(!iszero, pi[1:N])
    N1 = length(presence_list)
    absence_list = findall(iszero, pi[1:N])
    MatA_i = M[1:N, :] - sparse(Matrix(1.0I, N, N))
    pi_list = findall(!isequal(presence_list[1]), 1:N)
    MatA_i[:, pi_list] .-= MatA_i[:, presence_list[1]] * transpose(pi[pi_list] / pi[presence_list[1]])
    
    xi_hat = sum(pi[1:N] .* xi[1:N])
    MatB_i = (xi_hat * ones(N) - xi[1:N]) * N1
    
    beta_i = zeros(Float64, N)
    beta_i[pi_list] = idrs(MatA_i[pi_list, pi_list], MatB_i[pi_list])
    beta_i[presence_list[1]] = -sum(pi[1:N] .* beta_i) / pi[presence_list[1]]

    # Solve for β_ij
    Mat1 = M[1:N, :]
    inds = findall(!iszero, Mat1)
    a, b = getindex.(inds, 1), getindex.(inds, 2)
    vec1 = transpose(1:N)
    
    X1 = N * (a .- 1) * ones(Int, 1, N) + ones(Int, length(a)) * vec1
    Y1 = N * (b .- 1) * ones(Int, 1, N) + ones(Int, length(b)) * vec1
    W1 = Mat1[inds] * ones(Int, 1, N)
    vec2 = transpose((0:N-1) * N)
    X2 = a * ones(Int, 1, N) + ones(Int, length(a)) * vec2
    Y2 = b * ones(Int, 1, N) + ones(Int, length(b)) * vec2
    W2 = Mat1[inds] * ones(Int, 1, N)

    MatA_ij = sparse(vec(X1), vec(Y1), vec(W1), N^2, N^2)/2 + 
              sparse(vec(X2), vec(Y2), vec(W2), N^2, N^2)/2 - sparse(1.0I, N^2, N^2)
    
    beta_obtained = [(i-1)*N + i for i = 1:N]
    beta_missed = setdiff(1:N^2, beta_obtained)
    
    MatB_ij = -sum(MatA_ij[:, beta_obtained] .* transpose(beta_i), dims=2)
    Mat = xi_hat * ones(N, N) * N1/2 - xi[1:N] * transpose(xi[1:N]) * N1/2
    Mat[absence_list, :] .= 0; Mat[:, absence_list] .= 0
    MatB_ij .+= vec(transpose(Mat))
    
    beta_ij = zeros(Float64, N^2)
    beta_ij[beta_missed] = idrs(MatA_ij[beta_missed, beta_missed], MatB_ij[beta_missed])
    beta_ij[beta_obtained] = beta_i

    solution = spzeros(Float64, N^2, L)
    solution[:, 1] = beta_ij
    
    # Solve for γ_ij (inter-layer) - abbreviated for space
    if L > 1
        for ell = 2:L
            xi_ell = xi[(ell-1)*N+1:ell*N]
            pi_ell = pi[(ell-1)*N+1:ell*N]
            presence_ell = findall(!iszero, pi_ell)
            N_ell = length(presence_ell)
            
            GammaA_ij = sparse(vec(X1), vec(Y1), vec(W1), N^2, N^2) * (N_ell-1)/(N1+N_ell-1)
            Mat_ell = M[(ell-1)*N+1:ell*N, :]
            inds_ell = findall(!iszero, Mat_ell)
            a_ell, b_ell = getindex.(inds_ell, 1), getindex.(inds_ell, 2)
            X_ell = a_ell * ones(Int, 1, N) + ones(Int, length(a_ell)) * vec2
            Y_ell = b_ell * ones(Int, 1, N) + ones(Int, length(b_ell)) * vec2
            W_ell = Mat_ell[inds_ell] * ones(Int, 1, N)
            GammaA_ij += sparse(vec(X_ell), vec(Y_ell), vec(W_ell), N^2, N^2) * (N1-1)/(N1+N_ell-1)
            
            vec_g1 = vec(ones(Int, N) * vec1)
            vec_g2 = vec(transpose(vec1) * ones(Int, 1, N))
            X_g1 = N*(a .- 1)*ones(Int, 1, N^2) + ones(Int, length(a))*transpose(vec_g1)
            Y_g1 = N*(b .- 1)*ones(Int, 1, N^2) + ones(Int, length(b))*transpose(vec_g2)
            W_g1 = Mat1[inds] * ones(Int, 1, N^2)
            vec_g3 = (vec_g1 .- 1) * N
            vec_g4 = (vec_g2 .- 1) * N
            X_g2 = a_ell*ones(Int, 1, N^2) + ones(Int, length(a_ell))*transpose(vec_g3)
            Y_g2 = b_ell*ones(Int, 1, N^2) + ones(Int, length(b_ell))*transpose(vec_g4)
            W_g2 = Mat_ell[inds_ell] * ones(Int, 1, N^2)
            GammaA_ij += sparse(vec(X_g1), vec(Y_g1), vec(W_g1), N^2, N^2) .*
                         sparse(vec(X_g2), vec(Y_g2), vec(W_g2), N^2, N^2) / (N1+N_ell-1)
            GammaA_ij -= sparse(1.0I, N^2, N^2)
            
            pi_t = pi[1:N] .* pi_ell
            pres_g = findall(!iszero, pi_t)
            del = (pres_g .- 1) * N + pres_g
            GammaA_ij[:, del] .-= GammaA_ij[:, del[1]] * transpose(pi[pres_g]) / pi[pres_g[1]]
            pi_list_g = findall(!isequal(del[1]), 1:N^2)
            
            xi_ell_hat = sum(pi_ell .* xi_ell)
            GammaB = (xi_hat * xi_ell_hat * ones(N, N) - xi[1:N] * transpose(xi_ell)) * N1*N_ell/(N1+N_ell-1)
            pres1 = findall(iszero, pi[1:N])
            pres_e = findall(iszero, pi_ell)
            GammaB[pres1, :] .= 0; GammaB[:, pres_e] .= 0
            GammaB = vec(transpose(GammaB))
            
            gamma = zeros(Float64, N^2)
            gamma[pi_list_g] = idrs(GammaA_ij[pi_list_g, pi_list_g], GammaB[pi_list_g])
            gamma[del[1]] = -sum(pi[pres_g] .* gamma[del]) / pi[pres_g[1]]
            solution[:, ell] = gamma
        end
    end
    return solution
end

"""Compute θ,φ for death-Birth dynamics."""
function bc_multilayer_DB(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    edge_seq = edge_seq_rearrangement(edge_seq)
    solution = beta_gamma(edge_seq, xi_seq)
    N, L = maximum(edge_seq[:,1]), maximum(edge_seq[:,4])
    
    M = spzeros(Float64, N*L, N)
    pi = spzeros(Float64, L*N)
    xi = spzeros(Float64, L*N)
    
    for ell = 1:L
        idx = (ell-1)*N+1:ell*N
        edge_list = findall(isequal(ell), edge_seq[:,4])
        M[idx, :] = sparse(edge_seq[edge_list,1], edge_seq[edge_list,2], edge_seq[edge_list,3], N, N)
        pi_2 = sum(M[idx, :], dims=2)[:,1]
        pres = findall(!iszero, pi_2)
        pi_t = spzeros(Float64, N); pi_t[pres] = pi_2[pres] / sum(pi_2)
        pi[idx] = pi_t
        M[idx, :][pres, :] ./= pi_2[pres]
        xi_t = spzeros(Int64, N)
        xi_pres = findall(isequal(ell), xi_seq[:,3])
        xi_t[xi_seq[xi_pres,1]] = xi_seq[xi_pres,2]
        xi[idx] = xi_t
    end

    beta = transpose(reshape(solution[:,1], N, :))
    M1 = M[1:N, :]
    theta_phi = zeros(Float64, 4, L)
    theta_phi[:,1] = [sum(pi[1:N].*xi[1:N]), sum(pi[1:N].*M1.*beta), 
                      sum(pi[1:N].*diag(beta)), sum(pi[1:N].*transpose(M1).*transpose(beta))]
    
    for ell = 2:L
        idx = (ell-1)*N+1:ell*N
        gamma = transpose(reshape(solution[:,ell], N, :))
        theta_phi[:,ell] = [sum(pi[idx].*xi[idx]), sum(pi[1:N].*transpose(M[idx,:]).*gamma.*gamma),
                            sum(pi[1:N].*diag(gamma)), sum(pi[1:N].*M[idx,:].*transpose(gamma).*gamma)]
    end
    return theta_phi
end

"""Compute θ,φ for fitness-proportional dynamics."""
function bc_multilayer_DB_ff(edge_seq::Array{Int64,2}, xi_seq::Array{Int64,2})
    # Same structure as bc_multilayer_DB but with modified correlation formula
    edge_seq = edge_seq_rearrangement(edge_seq)
    solution = beta_gamma(edge_seq, xi_seq)
    N, L = maximum(edge_seq[:,1]), maximum(edge_seq[:,4])
    
    M = spzeros(Float64, N*L, N)
    pi = spzeros(Float64, L*N)
    xi = spzeros(Float64, L*N)
    
    for ell = 1:L
        idx = (ell-1)*N+1:ell*N
        edge_list = findall(isequal(ell), edge_seq[:,4])
        M[idx, :] = sparse(edge_seq[edge_list,1], edge_seq[edge_list,2], edge_seq[edge_list,3], N, N)
        pi_2 = sum(M[idx, :], dims=2)[:,1]
        pres = findall(!iszero, pi_2)
        pi_t = spzeros(Float64, N); pi_t[pres] = pi_2[pres] / sum(pi_2)
        pi[idx] = pi_t
        M[idx, :][pres, :] ./= pi_2[pres]
        xi_t = spzeros(Int64, N)
        xi_pres = findall(isequal(ell), xi_seq[:,3])
        xi_t[xi_seq[xi_pres,1]] = xi_seq[xi_pres,2]
        xi[idx] = xi_t
    end

    beta = transpose(reshape(solution[:,1], N, :))
    M1 = M[1:N, :]
    theta_phi = zeros(Float64, 4, L)
    # Modified formula for ff rule
    theta_phi[:,1] = [sum(pi[1:N].*xi[1:N]), sum(pi[1:N].*M1.*beta.*transpose(M1)), 
                      sum(pi[1:N].*diag(beta)), sum(pi[1:N].*transpose(M1).*transpose(beta).*M1)]
    
    for ell = 2:L
        idx = (ell-1)*N+1:ell*N
        gamma = transpose(reshape(solution[:,ell], N, :))
        M_ell = M[idx, :]
        theta_phi[:,ell] = [sum(pi[idx].*xi[idx]), sum(pi[1:N].*transpose(M_ell).*gamma.*M1.*gamma),
                            sum(pi[1:N].*diag(gamma)), sum(pi[1:N].*M_ell.*transpose(gamma).*transpose(M1).*gamma)]
    end
    return theta_phi
end

---
## 3. Random Graph Generation

In [ ]:
"""Convert graph to edge sequence format."""
function get_edge_sequence(g::Graphs.SimpleGraph, layer_id::Int64)
    edges_list = collect(edges(g))
    edge_seq = Array{Int64,2}(undef, 2 * length(edges_list), 4)
    for (idx, e) in enumerate(edges_list)
        u, v = src(e), dst(e)
        edge_seq[2idx-1, :] = [u, v, 1, layer_id]
        edge_seq[2idx, :]   = [v, u, 1, layer_id]
    end
    return edge_seq
end

"""Generate strategy config with cooperators at nodes i (layer 1) and j (layer 2)."""
function generate_xi_seq(N::Int64, i::Int64, j::Int64)
    xi_seq = zeros(Int64, 2N, 3)
    for k in 1:N
        xi_seq[k, :] = [k, 0, 1]
        xi_seq[k+N, :] = [k, 0, 2]
    end
    xi_seq[i, 2] = 1
    xi_seq[j+N, 2] = 1
    return xi_seq
end

"""Generate connected ER graphs with caching."""
function generate_ER_graphs(N::Int, probs::Vector{Float64}, num_seeds::Int; filepath::String)
    isfile(filepath) && (println("Loading from $filepath"); return deserialize(filepath))
    
    graphs = Dict{Float64, Vector{Tuple{SimpleGraph{Int64}, UInt64}}}()
    for p in probs
        graphs[p] = []
        while length(graphs[p]) < num_seeds
            seed = rand(UInt); Random.seed!(seed)
            g = erdos_renyi(N, p)
            is_connected(g) && push!(graphs[p], (g, seed))
        end
        println("  p=$p: $(length(graphs[p])) graphs")
    end
    serialize(filepath, graphs)
    return graphs
end

"""Generate connected BA graphs with caching."""
function generate_BA_graphs(N::Int, k_vals::Vector{Int}, num_seeds::Int; filepath::String)
    isfile(filepath) && (println("Loading from $filepath"); return deserialize(filepath))
    
    graphs = Dict{Int, Vector{Tuple{SimpleGraph{Int64}, Int64}}}()
    for k in k_vals
        graphs[k] = []
        while length(graphs[k]) < num_seeds
            seed = rand(Int64); Random.seed!(seed)
            g = barabasi_albert(N, k)
            is_connected(g) && push!(graphs[k], (g, seed))
        end
        println("  k=$k: $(length(graphs[k])) graphs")
    end
    serialize(filepath, graphs)
    return graphs
end

---
## 4. Simulation Engine

In [ ]:
"""Main simulation: compute θ,φ for all graph pairs and cooperator placements."""
function run_simulation(; graph_type::Symbol, update_rule::Symbol, N::Int, 
                         params::Vector, num_seeds::Int, checkpoint_every::Int)
    solver = update_rule == :dB ? bc_multilayer_DB : bc_multilayer_DB_ff
    prefix = lowercase("$(graph_type)_$(update_rule)")
    graphs_path = "graphs_$(prefix)_$(N).dat"
    theta_path = "theta_$(prefix)_$(N).dat"
    
    # Load or generate graphs
    graphs = graph_type == :ER ? 
        generate_ER_graphs(N, Float64.(params), num_seeds; filepath=graphs_path) :
        generate_BA_graphs(N, Int.(params), num_seeds; filepath=graphs_path)
    
    # Load or init results
    theta_dict = isfile(theta_path) ? deserialize(theta_path) : Dict{Tuple, Matrix{Float64}}()
    
    # Build graph list
    graph_list = [(p, g, s) for p in params for (g, s) in graphs[p]]
    total = length(graph_list)^2
    println("[$(uppercase(string(graph_type)))] Starting: $total pairs")
    
    pair_idx = 0
    for (p1, g1, s1) in graph_list, (p2, g2, s2) in graph_list
        pair_idx += 1
        edge_seq = vcat(get_edge_sequence(g1, 1), get_edge_sequence(g2, 2))
        
        new_ct = 0
        for i in 1:N, j in 1:N
            key = (Float64(p1), Float64(p2), i, j, s1, s2)
            if !haskey(theta_dict, key)
                theta_dict[key] = solver(edge_seq, generate_xi_seq(N, i, j))
                new_ct += 1
            end
        end
        
        @printf("[%4d/%d] p1=%.2f p2=%.2f | +%d new | total=%d\n", pair_idx, total, p1, p2, new_ct, length(theta_dict))
        pair_idx % checkpoint_every == 0 && (serialize(theta_path, theta_dict); GC.gc())
    end
    
    serialize(theta_path, theta_dict)
    println("Done. Saved to $theta_path")
    return theta_dict
end

---
## 5. Classification & Analysis

In [ ]:
"""Classify cooperation outcome based on θ,φ values."""
function classify_kind(θϕ; eps=1e-10)
    θ1, θ2, θ3 = θϕ[2,1], θϕ[3,1], θϕ[4,1]
    ϕ20 = θϕ[3,2]
    d = θ1 - θ3
    
    d > eps && return (:TRUE, :R1_dpos)
    d < -eps && ϕ20 >= 0 && return (:TRUE, :R2_dneg_phi20_nonneg)
    d < -eps && ϕ20 < 0 && return (-θ2/d - ϕ20/d > 0 ? (:TRUE, :R3) : (:FALSE, :F_R3))
    ϕ20 < 0 && return (:TRUE, :R4_phi20_neg)
    abs(ϕ20) <= eps && return (θ2 > 0 ? (:TRUE, :R4_theta2_pos) : (:FALSE, :F_R4))
    return (θ2 > -ϕ20 ? (:TRUE, :R4) : (:FALSE, :F_R4))
end

"""Generate summary statistics."""
function analyze_results(theta_dict::Dict)
    pair_totals = Dict{Tuple{Float64,Float64}, Int}()
    pair_true = Dict{Tuple{Float64,Float64}, Int}()
    
    for ((p1, p2, _, _, _, _), θϕ) in theta_dict
        pair_totals[(p1,p2)] = get(pair_totals, (p1,p2), 0) + 1
        classify_kind(θϕ)[1] == :TRUE && (pair_true[(p1,p2)] = get(pair_true, (p1,p2), 0) + 1)
    end
    
    println("\n" * "="^50)
    println("SUMMARY: $(sum(values(pair_true)))/$(sum(values(pair_totals))) cooperation favored")
    println("="^50)
    
    for (p1, p2) in sort(collect(keys(pair_totals)))
        t, tr = pair_totals[(p1,p2)], get(pair_true, (p1,p2), 0)
        @printf("  p1=%.1f, p2=%.1f: %d/%d (%.1f%%)\n", p1, p2, tr, t, 100tr/t)
    end
end

---
## 6. Execute

In [ ]:
# Run based on configuration
params = GRAPH_TYPE == :ER ? ER_PROBS : BA_K_VALUES

results = run_simulation(
    graph_type = GRAPH_TYPE,
    update_rule = UPDATE_RULE,
    N = N,
    params = params,
    num_seeds = NUM_SEEDS,
    checkpoint_every = CHECKPOINT_EVERY
)

In [ ]:
analyze_results(results)